## Практическое задание к уроку 3 по теме "Embedding word2vec fasttext".

Задача поиск похожих по эмбеддингам  

Скачиваем датасет (источник): положительные, отрицательные.  

или можно через ноутбук  
  
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv  
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv  

что надо сделать  
1. объединить в одну выборку  
2. на основе word2vec/fasttext/glove/слоя Embedding реализовать метод поиска ближайших твитов  
(на вход метода должен приходить запрос (какой-то твит, вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших твитов к этому запросу)  
3. Проверить насколько хорошо работают подходы

Загрузим необходимые библиотеки и датасеты:

In [1]:
import annoy
from gensim.models import FastText
from multiprocessing import Pool
from nltk.tokenize import TweetTokenizer
import numpy as np
import pandas as pd
from pymorphy2 import MorphAnalyzer
import re
from stop_words import get_stop_words
from string import punctuation
from tqdm import tqdm
import urlextract

In [2]:
RANDOM_STATE = 29
NUM_THREADS = 12

In [3]:
df_pos = pd.read_csv('../../Теория/Lesson_2/positive.csv', sep=';', header=None, usecols=[3], names=['tweet'])
df_neg = pd.read_csv('../../Теория/Lesson_2/negative.csv', sep=';', header=None, usecols=[3], names=['tweet'])
df = pd.concat((df_pos, df_neg), axis=0)
df.shape

(226834, 1)

In [4]:
df.head()

,tweet
0,"@first_timee хоть я и школота, но поверь, у на..."
1,"Да, все-таки он немного похож на него. Но мой ..."
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...
3,"RT @digger2912: ""Кто то в углу сидит и погибае..."
4,@irina_dyshkant Вот что значит страшилка :D\nН...


Загрузим стопслова, добавим туда часто встречающееся слово rt, которое субъективно  
не несёт смысла:

In [5]:
sw = get_stop_words('russian')
sw.append('rt')

Будем чистить от пунктуации, но оставим знаки, которые могут  
помешать работе нашего токенайзера, заточенного на работу с твитами:

In [6]:
punct = re.sub('[@_]', '', punctuation)

Будем искать и убирать ссылки, так как они могут помешать работе  
модели:

In [7]:
url_extractor = urlextract.URLExtract()

Берём nltk токенайзер по твитам, лемматизацию будем проводить  
с помощью pymorphy2:

In [8]:
tokenizer = TweetTokenizer(reduce_len=True, strip_handles=True)
lemmatizer = MorphAnalyzer()

Напишем функцию для обработки наших твитов:

In [9]:
def process_text(text):
    
    # Чистим ссылки
    urls = url_extractor.find_urls(text, only_unique=True)
    for url in urls:
        text = text.replace(url, ' ')
        
    # Убираем смайлик с буквой, который может остаться
    # после чистки от пунктуации, и приводим к нижнему
    # регистру
    text = re.sub(':D+', ' ', text).lower()
    
    # Чистим пунктуацию
    text = re.sub(fr'[{punct}]+', ' ', text)
    
    # Убираем всё, не являющееся набором букв, в т.ч. цифры
    text = ' '.join(word for word in text.split() if word.isalpha())
    
    # Токенизация
    text = tokenizer.tokenize(text)
    
    # Лемматизация
    text = [lemmatizer.parse(word)[0].normal_form for word in text]
    
    # Убираем стопслова
    text = [word for word in text if word not in sw]
    
    # Возвращаем обработанный текст
    return text

Обрабатываем текст:

In [10]:
with Pool(NUM_THREADS) as p:
    df['processed_tweet'] = tqdm(p.imap(process_text, df['tweet']), total=len(df))

100%|█████████████████████████████████| 226834/226834 [01:05<00:00, 3439.22it/s]


In [11]:
df.head()

,tweet,processed_tweet
0,"@first_timee хоть я и школота, но поверь, у на...","[школотый, поверь, самый, общество, профилиров..."
1,"Да, все-таки он немного похож на него. Но мой ...","[таки, похожий, мальчик, равно, хороший]"
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,"[идиотка, испугаться]"
3,"RT @digger2912: ""Кто то в углу сидит и погибае...","[угол, сидеть, погибать, голод, порция, взять,..."
4,@irina_dyshkant Вот что значит страшилка :D\nН...,"[страшилка, блин, посмотреть, часть, создаться..."


Обучаем модель FastText, т.к. в твитах большая вероятность появления  
опечаток, а эта модель обучается также и на n-граммах и имеет некоторую  
устойчивость к опечаткам:

In [12]:
%%time

ft_model = FastText(df['processed_tweet'], vector_size=300, window=3, min_count=2, 
                    seed=RANDOM_STATE, workers=NUM_THREADS, epochs=20)

CPU times: user 8min 16s, sys: 1.13 s, total: 8min 17s
Wall time: 54.7 s


Напишем функцию для получения вектора твитов через векторы слов.  
Будем просто находить среднее всех векторов слов твита, имеющихся  
в словаре модели, либо возвращать нулевой вектор, если ни одного  
слова нет:

In [13]:
def get_sentence_vector(sentence, model):
    counter = 0
    vec = np.zeros(model.vector_size)
    for word in sentence:
        if word in model.wv:
            vec += model.wv[word]
            counter += 1
    return vec / counter if counter else vec

На основании векторов твитов построим деревья annoy:

In [14]:
%%time

ft_index = annoy.AnnoyIndex(ft_model.vector_size, 'angular')

for i, sentence in enumerate(df['processed_tweet']):
    ft_vec = get_sentence_vector(sentence, ft_model)
    ft_index.add_item(i, ft_vec)

ft_index.build(20)

CPU times: user 29.6 s, sys: 207 ms, total: 29.8 s
Wall time: 10.7 s


True

Находить близкие твиты будем с помощью annoy, напишем функцию:

In [15]:
def get_similar_tweets(tweet, model, k=5):
    tweet = process_text(tweet)
    tweet_vec = get_sentence_vector(tweet, model)
    idxs = ft_index.get_nns_by_vector(tweet_vec, k)
    return df['tweet'].iloc[idxs].values

Напишем произвольные предложения и проверим на них работу модели:

In [16]:
texts = ['Я просто не знаю, что дальше делать!',
         'Сколько нужно денег для счастья?',
         'Почему ты мне не звонишь?',
         'Я сегодня купил чёрствый хлеб!',
         'Делаешь плохо - получается плохо! Делаешь хорошо - получается хорошо!'
        ]

In [17]:
print('*' * 50)
for i, text in enumerate(texts):
    print(f'{i+1} {text}')
    print('*' * 50)
    print(get_similar_tweets(text, ft_model, k=5))
    print('*' * 50, end='\n\n')

**************************************************
1 Я просто не знаю, что дальше делать!
**************************************************
['RT @SemenSemin: Теперь будем знать, как это делать))\nhttps://t.co/iRZJExZ9kq'
 'ну вы знаете , что делать :))) http://t.co/PjRasgg62q'
 'Спасибо тебе, @Kovalskiyyy, не знаю, чтобы я без тебя делала бы:*'
 '@kamilla_kub Только не знаю делают на мой или нет(('
 'не знаю, что теперь делать, а было все так хорошо(']
**************************************************

2 Сколько нужно денег для счастья?
**************************************************
['@pechNnik @Pensionnik Не в деньгах счастье=('
 'RT @385842387: не в деньгах счастье, а в их количестве!:)'
 '@gurodynawyr, Если ваше счастье не в деньгах, шлите их мне)'
 '@kolekcijaEDA камень - это разве подарок?)не в деньгах счастье)'
 'Почему у всех есть деньги на веселье, а у меня нет?:(']
**************************************************

3 Почему ты мне не звонишь?
***************************

В целом, модель улавливает похожие тексты, но, к примеру, в 4 примере модель  
"поняла", что такое "покупка", но "не понимает", что такое "чёрствый хлеб".  
Скорее всего из-за того, что не встречала этого на обучении.